<div class="alert alert-block alert-info">
<h3>Student Information</h3> Please provide information about yourself.<br>
<b>Name</b>:<br>
<b>NetID</b>:<br>
<b>Recitation (01/02)</b>:<br>
<b>Notes to Grader</b> (optional):<br>
<br><br>
<b>IMPORTANT</b>
Your work will not be graded withour your initials below<br>
I certify that this lab represents my own work and I have read the RU academic intergrity policies at<br>
<a href="https://www.cs.rutgers.edu/academic-integrity/introduction">https://www.cs.rutgers.edu/academic-integrity/introduction </a><br>
<b>Initials</b>:     


<h3>Grader Notes</h3>
<b>Your Grade<b>:<br>
<b>Grader Initials</b>:<br>
<b>Grader Comments</b> (optional):<br>
</div>

# Lab 7: Linear Models, Feature Engineering and Logistic Regression

** This lab is due Monday April 22, 2019 at 11:59pm (graded on accuracy and completeness) **

In this lab we will work through the process of:
1. implementing a linear model, defining loss functions, 
2. feature engineering,
3. minimizing loss functions using numeric libraries 
4. how to implement logistic regression model for a binary classification problem

We will use the toy tip calculation dataset from sns.

## Set up

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
np.random.seed(42)
plt.style.use('fivethirtyeight')
sns.set()
sns.set_context("talk")
%matplotlib inline

## Load the Tips Dataset

To begin with, we load the tips dataset from the `seaborn` library.  The tips data contains records of tips, total bill, and information about the person who paid the bill.

In [2]:
data = sns.load_dataset("tips")
print("Number of Records:", len(data))
data.head()

Number of Records: 244


,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


# Part 1: Defining the Model and Feature Engineering

In lab 6, we defined a simple linear model with only one parameter. Now let's make a more complicated model that utilizes other features in a dataset. Let our prediction for tip be a combination of the following features:

$$
\text{Tip} = \theta_1 * \text{total_bill} + \theta_2 * \text{sex} + \theta_3 * \text{smoker} + \theta_4 * \text{day} + \theta_5 * \text{time} + \theta_6 * \text{size}
$$

Notice that some of these features are not numbers! But our linear model will need to predict a numerical value. 
### Task 1.1 Split the data into Y(tip) and X(all others)

In [3]:
## BEGIN SOLUTION
tips = data['tip'].to_frame()
X = data.copy().drop(columns = ['tip'])
## END SOLUTION

## Task 1.2: Feature Engineering
First, let's convert everything to numerical values. A straightforward approach is to map some of these non-numerical features into numerical ones. For example, we can treat the day as a value from 1-7. However, one of the disadvantages in directly translating to a numeric value is that we unintentially assign certain features disproportionate weight. Consider assigning Sunday to the numeric value of 7. Monday is assigned to 1 and thus Sunday has 7 times the influence of Monday in our linear model which can lower the accuracy of our model.

solution: one-hot encoding! 

As discussed in lecture 9.2, one-hot encoding will produce a binary vector indicating the non-numeric feature. Sunday would be encoded as a [0 0 0 0 0 0 1]. This assigns a more even weight across each category in non-numeric features. Complete the code below to one-hot encode our dataset.

In [4]:
def one_hot_encode(data):
    # BEGIN SOLUTION
    return pd.get_dummies(data)
    # END SOLUTION
one_hot_X = one_hot_encode(X)
one_hot_X = one_hot_X[[ 'size','total_bill', 'day_Thur', 'day_Fri', 'day_Sat', 'day_Sun','sex_Male', 'sex_Female', 'smoker_Yes',
       'smoker_No', 'time_Lunch', 'time_Dinner']]
one_hot_X.head()

,size,total_bill,day_Thur,day_Fri,day_Sat,day_Sun,sex_Male,sex_Female,smoker_Yes,smoker_No,time_Lunch,time_Dinner
0,2,16.99,0,0,0,1,0,1,0,1,0,1
1,3,10.34,0,0,0,1,1,0,0,1,0,1
2,3,21.01,0,0,0,1,1,0,0,1,0,1
3,2,23.68,0,0,0,1,1,0,0,1,0,1
4,4,24.59,0,0,0,1,0,1,0,1,0,1


## Task 1.2: Defining the Model
Now that all of our data is numeric, let's define our model function. Note that X and thetas are matrices now. Use matrix products to compute the model.

In [5]:
def linear_model(thetas, X):
    """
    Return the linear combination of thetas and features as defined above.
    """
    return X.dot(thetas)
    ### END SOLUTION

In [6]:
assert linear_model(np.arange(1,5), np.arange(1,5)) == 30

# Part 2: Fitting the Model: Numeric Methods
Recall in the previous lab we defined multiple loss functions and found optimal theta using the scipy.minimize function. Adapt the loss functions and optimization code from the previous lab (provided below) to work with your new linear model.

In [7]:
from scipy.optimize import minimize

def abserror(y, y_hat):
    return np.abs(y-y_hat)

def sqerror(y, y_hat):
    return (y - y_hat)**2

def minimize_average_loss(loss_function, model, x, y):
    """
    loss_function: either the squared or absolute loss functions from above.
    model: the model (as defined above)
    x: the x values (one-hot encoded data)
    y: the y values (tip amounts)
    return the estimate for each theta as a vector
    
    Note we will ignore failed convergence for this lab ... 
    """
    x = x.values
    y = y.values
   
    return minimize(lambda theta: loss_function(model(theta, x), y).mean(), x0=np.random.rand(x.shape[1]))['x']
minimize_average_loss(sqerror, linear_model, one_hot_X, tips)

array([-7.57999725e-07, -6.68735440e-08,  5.50042386e-01,  5.50038087e-01,
        5.50048817e-01,  5.50047581e-01,  7.40882618e-01,  7.40888254e-01,
        9.33347721e-01,  9.33350831e-01,  7.74004011e-01,  7.74001416e-01])

# Part 3: Fitting the Model: Analytic Methods
Let's also fit our model analytically, for the l2 loss function. In this question we will derive an analytical solution, fit our model and compare our results with our numerical optimization results.

## Task 3.1: Least Squares Solution
Recall that if we're fitting a linear model with the l2 loss function, we are performing least squares! Remember, we are solving the following optimization problem for least squares:

$$\min_{\theta} ||X\theta - y||^2$$

Let's begin by deriving the analytic solution to least squares. Write your answer in LaTeX in the cell below. Assume X is full column rank. You are given step 1 to get things started. You need to complete the steps 2-4 where step 4 shows the value of theta that optimizes given X and y.

### BEGIN SOLUTION
Take the gradient and set theta equal to 0. 

$$2(X\theta - y) = 0$$
$$ step 2$$
$$ step 3$$
$$ \theta = step 4$$
### END SOLUTION

## Task 3.2: Solving for Theta
Using your formuala above, let us find the analytic solution for $\theta$. That is, the optimal numerical thetas for our tips dataset. Fill out the function below. Make sure you use the float type in your calculations using .astype(float) and use the np.linalg.inv function.

In [8]:
def get_analytical(x, y):
    """
    x: our one-hot encoded dataset
    y: tip amounts
    """
    x = x.values
    y = y.values
    xTx = x.T.dot(x)
    xTy = x.T.dot(y)
    return np.linalg.inv(xTx).dot(xTy)
    ### END SOLUTION

In [9]:
analytical_thetas = get_analytical(one_hot_X.astype(float), tips.astype(float))
print("Our analytical loss is: ", sqerror(linear_model(analytical_thetas, one_hot_X).values,tips).values.mean())
print("Our numerical loss is: ", sqerror(linear_model(minimize_average_loss(sqerror, linear_model, one_hot_X, tips), one_hot_X).values, tips.values).mean())

Our analytical loss is:  246.45677797131148
Our numerical loss is:  1.9066085126651786


In [10]:
assert np.isclose(sqerror(linear_model(analytical_thetas, one_hot_X).values,tips).values.mean(), 246.45677)

## Task 3.3: Weird Results?
Our analytical loss is surprisingly much worse than our numerical loss. Why is this? 

Hint: https://stackoverflow.com/questions/31256252/why-does-numpy-linalg-solve-offer-more-precise-matrix-inversions-than-numpy-li

In summary, np.linalg.inv loses precision, which propogates error throughout the calculation. If you're not convinced, try np.linalg.solve instead of np.linalg.inv, you'll find that our loss is much closer to the expected numerical loss. These results are meant to demonstrate that even if our math is correct, limits of our computational precision and machinery can lead us to poor results.

### BEGIN SOLUTION



### END SOLUTION

# Part 4 - Logistic Regression
In this part we will implement a logistic regression model with the tip data set. We will use Male and Female as the binary classification. We will use the LogisiticRegression model from sklearn to find a prediction model.

## task 4.1
** create the X (one-hot features) and y (sex) **

In [11]:
## BEGIN SOLTUION
X = one_hot_X.copy().drop(columns = ['sex_Male', 'sex_Female'])
y = data['sex']
## END SOLUTION
print(X.head())
print(y.head())


   size  total_bill  day_Thur  day_Fri  day_Sat  day_Sun  smoker_Yes  \
0     2       16.99         0        0        0        1           0   
1     3       10.34         0        0        0        1           0   
2     3       21.01         0        0        0        1           0   
3     2       23.68         0        0        0        1           0   
4     4       24.59         0        0        0        1           0   

   smoker_No  time_Lunch  time_Dinner  
0          1           0            1  
1          1           0            1  
2          1           0            1  
3          1           0            1  
4          1           0            1  
0    Female
1      Male
2      Male
3      Male
4    Female
Name: sex, dtype: category
Categories (2, object): [Male, Female]


## task 4.2 
Split the data set to training and test using sklearn train_test_split

In [12]:
from sklearn.model_selection import train_test_split
## BEGIN SOLUTION
X_train, X_test, y_train, y_test = train_test_split(X,y)
## END SOLUTION

print (X_train.head(), X_test.head(), y_train.head(), y_test.head())

     size  total_bill  day_Thur  day_Fri  day_Sat  day_Sun  smoker_Yes  \
90      2       28.97         0        1        0        0           1   
37      3       16.93         0        0        1        0           0   
144     2       16.43         1        0        0        0           0   
16      3       10.33         0        0        0        1           0   
139     2       13.16         1        0        0        0           0   

     smoker_No  time_Lunch  time_Dinner  
90           0           0            1  
37           1           0            1  
144          1           1            0  
16           1           0            1  
139          1           1            0        size  total_bill  day_Thur  day_Fri  day_Sat  day_Sun  smoker_Yes  \
30      2        9.55         0        0        1        0           0   
119     4       24.08         1        0        0        0           0   
193     2       15.48         1        0        0        0           1   
208    

## task 4.3 logistic regression model
Fit a logisitic regression model using LogisticRegression from sklearn

In [13]:
from sklearn.linear_model import LogisticRegression
## BEGIN SOLUTION
logmodel = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial').fit(X_train, y_train)

## END SOLUTION

## task 4.4 Predicting
Now we can use the logmodel from previous part to predict on the test data. Read documentation to see how. It should be just a one line of code.

In [14]:
## BEGIN SOLUTION
predictions = logmodel.predict(X_test)
## END SOLUTION

## task 4.5 reporting
use classification_report from sklearn.metrics to report the accuracy of the model.

In [15]:
from sklearn.metrics import classification_report
## BEGIN SOLUTION
y1 = y_test.to_frame().values.flatten()
print(classification_report(y1, predictions))
## END SOLUTION

              precision    recall  f1-score   support

      Female       0.55      0.22      0.32        27
        Male       0.58      0.85      0.69        34

   micro avg       0.57      0.57      0.57        61
   macro avg       0.56      0.54      0.50        61
weighted avg       0.56      0.57      0.52        61



### Feedback
Please provide feedback on this lab.
* how would you rate this lab (from 1-10, 10-highest) :
* how can we improve his lab? :

<div class="alert alert-block alert-info">
<h2>Submission Instructions</h2> 
<b> File Name:</b> Please name the file as your_section_your_netID_lab7.jpynb<br>
<b> Submit To: </b> Canvas &rarr; Assignments &rarr; lab7 <br>
<b>Warning:</b> Failure to follow directions may result in loss points.<br>
</div>

Developed by A.D. Gunawardena. Credits: Josh Hug, Berkeley Data Science Lab